In [1]:
# 파일 직접 업로드
from google.colab import files
fake_review_datas = files.upload()

Saving fake reviews dataset.csv to fake reviews dataset.csv


In [2]:
import torch
import pandas as pd
import numpy as np

In [3]:
import io

df = pd.read_csv(io.BytesIO(fake_review_datas['fake reviews dataset.csv']))
print(df.shape)
df.head()

(40432, 4)


category  rating label  \
0  Home_and_Kitchen_5     5.0    CG   
1  Home_and_Kitchen_5     5.0    CG   
2  Home_and_Kitchen_5     5.0    CG   
3  Home_and_Kitchen_5     1.0    CG   
4  Home_and_Kitchen_5     5.0    CG   

                                               text_  
0  Love this!  Well made, sturdy, and very comfor...  
1  love it, a great upgrade from the original.  I...  
2  This pillow saved my back. I love the look and...  
3  Missing information on how to use it, but it i...  
4  Very nice set. Good quality. We have had the s...

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
# tokenize

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
ps = PorterStemmer()
corpus = []
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['text_'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [6]:
for i in range(5):
    print(corpus[i])

love well made sturdi comfort love pretti
love great upgrad origin mine coupl year
pillow save back love look feel pillow
miss inform use great product price
nice set good qualiti set two month


In [7]:
# build vocab dictionary

import os
from collections import Counter

vocabs = [vocab for seq in corpus for vocab in seq.split()]
# vocabs = ['love', 'well', 'made', 'sturdi', 'comfort', 'love', 'pretti', 'love', 'great', ... ]  (corpus[:5] 했을때 기준)

# Count words in the whole Corpus
vocab_count = Counter(vocabs)
# vocab_count = Counter({'love': 4, 'great': 2, 'pillow': 2, 'set': 2, 'well': 1, 'made': 1, ... ] (corpus[:5] 했을때 기준)

# ordering words by their counts
vocab_count = vocab_count.most_common(len(vocab_count))

vocab_to_int = {word : index+2 for index, (word, count) in enumerate(vocab_count)}
vocab_to_int.update({'__PADDING__': 0}) # index 0 for padding
vocab_to_int.update({'__UNKNOWN__': 1}) # index 1 for unknown word such as broken character

# 가장 빈도수 높은 단어(어근)순
print(vocab_to_int)

{'love': 2, 'book': 3, 'great': 4, 'one': 5, 'good': 6, 'like': 7, 'use': 8, 'read': 9, 'well': 10, 'stori': 11, 'would': 12, 'work': 13, 'get': 14, 'littl': 15, 'charact': 16, 'recommend': 17, 'look': 18, 'time': 19, 'movi': 20, 'realli': 21, 'bought': 22, 'nice': 23, 'also': 24, 'product': 25, 'fit': 26, 'size': 27, 'small': 28, 'easi': 29, 'go': 30, 'year': 31, 'enjoy': 32, 'lot': 33, 'make': 34, 'qualiti': 35, 'bit': 36, 'dog': 37, 'want': 38, 'first': 39, 'made': 40, 'seri': 41, 'way': 42, 'two': 43, 'light': 44, 'much': 45, 'thing': 46, 'keep': 47, 'put': 48, 'price': 49, 'need': 50, 'review': 51, 'end': 52, 'buy': 53, 'set': 54, 'think': 55, 'purchas': 56, 'comfort': 57, 'see': 58, 'still': 59, 'author': 60, 'problem': 61, 'perfect': 62, 'old': 63, 'long': 64, 'anyon': 65, 'happi': 66, 'order': 67, 'feel': 68, 'say': 69, 'come': 70, 'interest': 71, 'take': 72, 'tri': 73, 'play': 74, 'color': 75, 'best': 76, 'know': 77, 'got': 78, 'plastic': 79, 'wear': 80, 'part': 81, 'better': 

In [8]:
from torch.autograd import Variable

# Tokenize & Vectorize sequences
vectorized_seqs = []
for seq in corpus: 
  vectorized_seqs.append([vocab_to_int.get(word,1) for word in seq.split()])

# Save the lengths of sequences
seq_lengths = torch.LongTensor(list(map(len, vectorized_seqs)))

# Add padding(0)
seq_tensor = Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long()
for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
  seq_tensor[idx, :seqlen] = torch.LongTensor(seq)

print(seq_lengths.max())
print(seq_tensor[0])
print(seq_lengths[0])

tensor(354)
tensor([  2,  10,  40, 132,  57,   2,  88,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,

In [ ]:
# Build Data loader